In [1]:
import os; print(os.getcwd())
import socket; print(socket.gethostname())
import os
from glob import glob

/home/aljubetic/AF2/projects/2023-03-14__mALB8_all_by_all
headnode


In [2]:
#import sys; sys.path.insert(0, "/home/ajasja/projects/Stable_Heterodimers")
#import utils as u
#import sys; sys.path.insert(0, "/home/ajasja/projects/truncator/")
import truncator
from truncator import pymol_utils
import truncator as u

from utils import *

In [4]:
out_dir = 'out/01__multimer-v3'
af2 = pd.read_csv(f'{out_dir}/AF2.csv.gz', index_col=0)

In [5]:
%%time

rosetta_subdir = 'rosetta-min-sc'
pdb_files=[]
score_files=[]

pdb_files.extend(sorted(glob(f"{out_dir}/{rosetta_subdir}/*/*.pdb")));
score_files.extend(sorted(glob(f"{out_dir}/{rosetta_subdir}/*/*.sc")));
print(f"PDB Files: {len(pdb_files)}")


#Replace unquoted nan values
#for scoref in score_files:
#    !sed -i -e 's/-nan/""/ag' -e 's/nan/""/g' {scoref}
scores = truncator.read_score_files_with_cache(score_files, f'{out_dir}/{rosetta_subdir}/rosetta_scores.csv.gz')
#convert to decoy naming. 
af2['decoy'] =  (af2.stem_name.str.replace('_scores_','_relaxed_')+'__rrelaxed_0001')
af2m = af2.merge(scores, on='decoy', how='left')
af2m['rosetta-protocol'] = rosetta_subdir
af2m.to_csv(f'{out_dir}/{rosetta_subdir}/AF2_rosetta.csv.gz')


PDB Files: 2420
CPU times: user 672 ms, sys: 107 ms, total: 780 ms
Wall time: 2.02 s


In [6]:
rosetta_subdir = 'rosetta-flex-bb'
pdb_files=[]
score_files=[]

pdb_files.extend(sorted(glob(f"{out_dir}/{rosetta_subdir}/*/*.pdb")));
score_files.extend(sorted(glob(f"{out_dir}/{rosetta_subdir}/*/*.sc")));
print(f"PDB Files: {len(pdb_files)}")


#Replace unquoted nan values
#for scoref in score_files:
#    !sed -i -e 's/-nan/""/ag' -e 's/nan/""/g' {scoref}
scores = truncator.read_score_files_with_cache(score_files, f'{out_dir}/{rosetta_subdir}/rosetta_scores.csv.gz')
#convert to decoy naming. 
af2['decoy'] =  (af2.stem_name.str.replace('_scores_','_relaxed_')+'__rrelaxed_0001')
af2m = af2.merge(scores, on='decoy', how='left')
af2m['rosetta-protocol'] = rosetta_subdir
af2m.to_csv(f'{out_dir}/{rosetta_subdir}/AF2_rosetta.csv.gz')

PDB Files: 2420


In [6]:
# Merge all
all_files = glob("out/*/*/AF2_rosetta.csv.gz")
scores = [pd.read_csv(af, index_col=0) for af in all_files]
merged = pd.concat(scores)
merged.reindex()
merged.to_csv('data/AF2_rosetta_merged.csv.gz')